# Estudo de oportunidades para PBZPA

In [1]:
# helper function
from os import listdir
def dir_files(name: str, folder: str = None):
    if folder:
        files = [folder + e for e in listdir(folder) if (e[-4:] == ".xls") or (e[-4:] == ".csv")]
    else:
        files = [e for e in listdir() if (e[-4:] == ".xls") or (e[-4:] == ".csv")]
    print(f"{name}: {', '.join(files)}")
    return files

In [29]:
from pandas import read_excel
from pandas import read_csv

public_private = dir_files('Aeródromos públicos e privados', r'E:/Users/falam/IdeaProjects/Aerodromos/ANAC/Beta/public_private/')
aerodromos_publicos =  read_excel(public_private[1], sheet_name=0, header=[1,2])
aerodromos_privados = read_excel(public_private[0], sheet_name=0, header=[0,1])

aisweb = dir_files('AISWeb')
aisweb =  read_csv(aisweb[0], sep=',', encoding='latin-1')

aisweb.insert(loc=4, column="Categoria",value='')
icao_aeropublicos = aerodromos_publicos.iloc[:,0].values
icao_aeroprivados = aerodromos_privados.iloc[:,0].values
for i, icao in enumerate(aisweb.ICAO):
    if icao in icao_aeropublicos:
        aisweb.at[i, 'Categoria'] = 'Público'
    elif icao in icao_aeroprivados:
        aisweb.at[i, 'Categoria'] = 'Privado'

Aeródromos públicos e privados: E:/Users/falam/IdeaProjects/Aerodromos/ANAC/Beta/public_private/Aeródromos Privados_28MAR2019.xls, E:/Users/falam/IdeaProjects/Aerodromos/ANAC/Beta/public_private/Aeródromos Públicos_28MAR2019.xls
AISWeb: output_aisweb_4428c205cd.csv


In [3]:
aisweb.count()

timestamp    2630
ICAO         2630
CIAD         2569
Aeródromo    2568
Categoria    2630
Cidade       2568
UF           2568
Status       2630
NOTAM        2630
Debug        2630
dtype: int64

### Há valores nulos?

In [4]:
aisweb.isnull().sum()

timestamp     0
ICAO          0
CIAD         61
Aeródromo    62
Categoria     0
Cidade       62
UF           62
Status        0
NOTAM         0
Debug         0
dtype: int64

### Aeródromos não encontrados (fechado, localidade cancelada, Localidade reservada aguardando portaria da ANAC)

#### Aeródromos que constam na lista da ANAC, mas não foram encontrados no AISWeb

In [9]:
aero_naoencontrado = aisweb[aisweb.Status == "O aeródromo não foi encontrado"]
print(f"Total de aeródromos não encontrados: {aero_naoencontrado.ICAO.count()}")

Total de aeródromos não encontrados: 3


In [ ]:
aero_nao_encontrado

#### Localidade cancelada

In [13]:
aero_locanc = aisweb[aisweb.Status == "Localidade cancelada"]
print(f"Total de aeródromos com {filtro}: {aero_locanc.ICAO.count()}")

Total de aeródromos com Localidade cancelada: 3


#### Aeródromos com localidade reservada aguardando portaria da ANAC

In [16]:
aero_locres = aisweb[aisweb.Status == "Localidade reservada. Aguardando portaria da ANAC."]
print(f"Total de aeródromos com {filtro}: {aero_locres.ICAO.count()}")

Total de aeródromos com Localidade cancelada: 54


#### Aeródromos fechados até determinada data

In [30]:
filtro = "Aeródromo FECHADO até"
aero_fechado = aisweb[aisweb.Status.str.contains(filtro) == True]
print(f"Total de aeródromos fechados: {aero_fechado.ICAO.count()}")

Total de aeródromos fechados: 165


#### Aeródromos com dados faltando (estudar a fundo, estes podem estar iniciando atividade)

In [22]:
aero_null = aisweb[aisweb.Cidade.isnull()]
print(f"Total de aeródromos com valores nulos: {aero_null.ICAO.count()}")

Total de aeródromos com valores nulos: 62


## NOTAM
Aeródromos contendo a string "AD CLSD DEVIDO NAO APRESENTACAO DO PLANO BASICO DE ZONA DE PROTECAO"

In [24]:
filtro = "AD CLSD DEVIDO NAO APRESENTACAO DO PLANO BASICO DE ZONA DE PROTECAO"
ad_clsd_pbzpa = aisweb[aisweb.NOTAM.str.contains(filtro)]
print(f"Total de aeródromos com \"AD CLSD DEVIDO NAO APRESENTACAO DO PLANO BASICO DE ZONA DE PROTECAO\": {ad_clsd_pbzpa.ICAO.count()}")

Total de aeródromos com "AD CLSD DEVIDO NAO APRESENTACAO DO PLANO BASICO DE ZONA DE PROTECAO": 193


#### Fechados, aguardando INFRACEA | Novos Negócios agir!

In [34]:
filtro = ".*Revogado INFOTEMP [0-9]*/[0-9]*.{0,6}AD CLSD DEVIDO NAO APRESENTACAO DO PLANO BASICO DE ZONA DE PROTECAO.*"
oportunidades = ad_clsd_pbzpa[~ad_clsd_pbzpa.NOTAM.str.contains(filtro, regex=True)]
print(f"Total de prospects para novos negócios: {oportunidades.ICAO.count()}")

Total de prospects para novos negócios: 147


#### Revogado o fechamento

In [25]:
filtro = ".*Revogado INFOTEMP [0-9]*/[0-9]*.{0,6}AD CLSD DEVIDO NAO APRESENTACAO DO PLANO BASICO DE ZONA DE PROTECAO.*"
ad_clsd_pbzpa_rev = ad_clsd_pbzpa[ad_clsd_pbzpa.NOTAM.str.contains(filtro, regex=True)]
print(f"Total de aeródromos com \"AD CLSD\" revogado: {ad_clsd_pbzpa_rev.ICAO.count()}")

Total de aeródromos com "AD CLSD" revogado: 46


#### Revogado o fechamento, mas somente em missões de caráter emergencial
ou seja, candidatos para INFRACEA | Novos Negócios com visibilidade social, será que tem prioridade para os governos, mas eles não tem dinheiro para pagar. Ação de doação de empresas com interesses nas regiões para operar o aeroporto?

In [26]:
filtro = "CARATER EMERGENCIAL"
ad_clsd_pbzpa_rev_emg = ad_clsd_pbzpa_rev[ad_clsd_pbzpa_rev.NOTAM.str.contains(filtro)]
print(f"Total de aeródromos com \"AD CLSD\" revogado em caráter emergencial: {ad_clsd_pbzpa_rev_emg.ICAO.count()}")

Total de aeródromos com "AD CLSD" revogado em caráter emergencial: 22


#### Revogação do fechamento ratificada
situação inferida: gestão fraca? Estão desesperados esperando a INFRACEA | Aeroportos?

In [37]:
ad_clsd_pbzpa_rev_ne = ad_clsd_pbzpa_rev[ad_clsd_pbzpa_rev.ICAO.isin(aero_fechado.ICAO)]
print(f"Total de aeródromos com funcionamento turbulento: {ad_clsd_pbzpa_rev_ne.ICAO.count()}")

Total de aeródromos com funcionamento turbulento: 32


##### Exportar para excel

In [ ]:
#ad_clsd_pbzpa.to_excel('notam_ad_clsd_pbzpa.xlsx', index=False, na_rep='')